[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap07/cap07-main.ipynb)

# Capítulo 7 - Algoritmos com Modelo


## Configurações Iniciais

In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg #freeglut3

    !pip install gymnasium
    !pip install optuna

    # clone repository
    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

#clear_output()

In [ ]:
import random as rand
import gymnasium as gym
import numpy as np

from util.experiments import repeated_exec
from util.plot import plot_result, plot_multiple_results
from util.notebook import display_videos_from_path

from util.qtable_helper import evaluate_qtable, record_video_qtable

O código abaixo registra o ambiente `envs.Racetrack` no `gymnasium` associado com a string `Racetrack`. Além disso, a quantidade de passos por episódio foi limitada a 300.

Com isso, é possível criar o ambiente (junto com o wrapper que limita os passos por episódio) simplesmente, excutando:

```
    gym.make("Racetrack")
```

In [ ]:
from gymnasium.envs import register

#from envs import RacetrackEnv

register(
    id="Racetrack",  # Use a unique string ID for your environment
    entry_point="envs:RacetrackEnv",  # Specify the module and class name
    max_episode_steps=500,
)

In [ ]:
#gym.make("Racetrack")

## 1 - Q-Learning

O *Q-Learning* é um algoritmo **livre de modelo**, que vamos comparar com o próximo algoritmo, chamado *Dyna-Q* que é um algoritmo **com modelo**. 

Recomendamos acessar  o **código** do Q-Learning no `cap05/qlearning_sarsa.py` para relembrar.

Também vamos executar o Q-Learning, logo abaixo, para comparar com o desempenho do próximo algoritmo.

In [ ]:
from cap05.qlearning_sarsa import run_qlearning

In [ ]:
# escolha o ambiente descomentando uma das linhas abaixo
#ENV_NAME = 'Taxi-v3'
ENV_NAME = 'Racetrack'

LR = 0.3
GAMMA = 0.90
EPSILON = 0.1

#VERBOSE = True

In [ ]:
env = gym.make(ENV_NAME)

if ENV_NAME == "Taxi-v3":
    rmax = 10.0
    EPISODES = 700
else:
    rmax = 0.0
    EPISODES = 3_000

rewards1, qtable1 = run_qlearning(env, EPISODES, LR, GAMMA, EPSILON)
print("Últimos resultados: media =", np.mean(rewards1[-20:]), ", desvio padrao =", np.std(rewards1[-20:]))

In [ ]:
# Mostra um gráfico de passos x retornos não descontados acumulados
plot_result(rewards1, rmax, cumulative=False, window=30)

In [ ]:
evaluate_qtable(env, qtable1, 10, verbose=True);

In [ ]:
record_video_qtable(env, qtable1, length=500, folder='./videos-qlearn')

In [ ]:
display_videos_from_path('./videos-qlearn', speed=0.5)

## 2 - Dyna-Q

O *Dyna-Q* é um algoritmo **com modelo** que é uma extensão do *Q-Learning*. Compare os códigos para entender o Dyna-Q.

In [ ]:
def planning(model, planning_steps, Q, lr, gamma):
    all_s_a = list(model.keys())
    if len(all_s_a) < planning_steps:
        samples = rand.choices(all_s_a, k=planning_steps)
    else:
        samples = rand.sample(all_s_a, k=planning_steps)

    for s, a in samples:
        r, next_s, is_terminal = model[(s,a)]
        if is_terminal:
            V_next_s = 0
        else:
            V_next_s = np.max(Q[next_s])
        delta = (r + gamma * V_next_s) - Q[s,a]
        Q[s,a] = Q[s,a] + lr * delta

In [ ]:
# Esta é a política. Neste caso, escolhe uma ação com base nos valores da tabela Q, usando uma estratégia epsilon-greedy, 
# dividindo a probabilidade igualmente em caso de empates entre ações de valor máximo.
from util.qtable_helper import epsilon_greedy_random_tiebreak

In [ ]:
# Algoritmo Dyna Q
def run_dyna_q(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, planning_steps=5, verbose=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

    # inicializa a tabela Q
    Q = np.zeros(shape=(env.observation_space.n, num_actions))

    model = dict({})

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state, _ = env.reset()

        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:

            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy_random_tiebreak(Q, state, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            if terminated:
                # para estados terminais
                V_next_state = 0
                next_state = env.reset()
            else:
                # para estados não-terminais -- valor máximo (melhor ação)
                V_next_state = np.max(Q[next_state])

            # atualiza a Q-table / direct RL
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            # atualiza o modelo
            model[state,action] = (reward, next_state, done)

            # planejamento / indirect RL
            planning(model, planning_steps, Q, lr, gamma)

            sum_rewards += reward
            state = next_state

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 1000 passos, imprime informação sobre o progresso
        if verbose and ((i+1) % 1000 == 0):
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Step {i+1} Average Reward (last 100): {avg_reward:.3f}")

    state = env.reset()
    reward = 0

    return sum_rewards_per_ep, Q

In [ ]:
env = gym.make(ENV_NAME)

if ENV_NAME == 'Taxi-v3':
    rmax = 10.0
    EPISODES = 700
else:
    rmax = 0.0
    EPISODES = 3_000

rewards2, qtable2 = run_dyna_q(env, EPISODES, LR, GAMMA, EPSILON, planning_steps=10, verbose=True)
print("Últimos resultados: media =", np.mean(rewards2[-20:]), ", desvio padrao =", np.std(rewards2[-20:]))

In [ ]:
# Mostra um gráfico de passos x retornos não descontados acumulados
plot_result(rewards2, rmax, cumulative='no', window=30)

In [ ]:
evaluate_qtable(env, qtable2, 10, verbose=True);

In [ ]:
record_video_qtable(env, qtable2, length=500, folder='./videos-dynaq')

In [ ]:
display_videos_from_path("./videos-dynaq", speed=0.5)

## 3 - Experimentos Q-Learning x Dyna-Q

Nesta seção, você pode fazer experimentos no `Taxi-v3` ou no `Racetrack`, como preferir. (Porém, com o `Racetrack` é mais demorado).

Escolha o ambiente na célula de código abaixo.

Os experimentos têm duas partes:
- Na seção 3.1, vamos otimizar os parâmetros do Q-Learning
- Na seção 3.2, vamos rodar o Q-Learning e o Dyna-Q (ambos com os parâmetros do Q-Learning), para comparar


In [ ]:
# Escolha abaixo o ambiente
ENV_NAME_EXPERIMENT = "Racetrack"
#ENV_NAME_EXPERIMENT = "Taxi-v3"

### 3.1 - Otimizando Parâmetros do Q-Learning

In [ ]:
import optuna

Veja o scrip `cap07/optimize_params.py`, que faz uma otimização usando o `optuna`. 

Depois, carregue os resultados com o comando abaixo:

In [ ]:
study = optuna.load_study(storage="sqlite:///optuna_cap07.db",
                          study_name=f"qlearning_{ENV_NAME_EXPERIMENT}")

print(f"MELHORES PARÂMETROS PARA {ENV_NAME_EXPERIMENT}:")
print(study.best_params)

qlearn_params_t = study.best_params

Se não quiser rodar a otimização, você pode descomentar uma das linhas abaixo.

In [ ]:
# para o Racetrack
#qlearn_params_t = {'lr': 0.6975653481439976, 'epsilon': 0.05401583102169205, 'gamma': 0.5524696359810837}

# para o Taxi
#qlearn_params_t = {'lr': 0.9159141726489135, 'epsilon': 0.025509143159405598, 'gamma': 0.7577721224627403}

### 3.2 - Experimentos Comparativos

In [ ]:
environment = gym.make(ENV_NAME_EXPERIMENT)

if ENV_NAME_EXPERIMENT == "Taxi-v3":
    EPISODES = 1000
else:
    EPISODES = 4_000

RUNS = 5
AUTO_LOAD = True

results_t = []

results_t.append( repeated_exec(RUNS, f"Q-Learning ", run_qlearning, environment, EPISODES, **qlearn_params_t, auto_load=AUTO_LOAD) )
clear_output()

#plan_steps = 1
#results_t.append( repeated_exec(RUNS, f"Dyna-Q ({plan_steps} passo)", run_dyna_q, environment, EPISODES, **qlearn_params_taxi, planning_steps=plan_steps, auto_load=AUTO_LOAD) )
#clear_output()

plan_steps = 10
results_t.append( repeated_exec(RUNS, f"Dyna-Q ({plan_steps} passos)", run_dyna_q, environment, EPISODES, **qlearn_params_t, planning_steps=plan_steps, auto_load=AUTO_LOAD) )
clear_output()


In [ ]:
plot_multiple_results(results_t, cumulative='avg', x_log_scale=False)

In [ ]:
plot_multiple_results(results_t, cumulative='no', x_log_scale=False, window=100)

## 4 - Experimentos com "FrozenLake-v1"

Nesta seção, vamos fazer experimentos similares aos da seção 3, mas focando no ambiente `FrozenLake`.

Este ambiente tem como diferencial o fato de ser **não-determinístico**:
- uma ação tem 1/3 de chance de dar o resultado correto
- e 1/3 de fazer o agente mover em cada direção perpendicular à desejada.

Isso afeta um pouco os resultados, como veremos.

In [ ]:
ENV_NAME_EXPERIMENT = "FrozenLake-v1"

Primeiramente, execute o script `cap07/optimize_params`, ajustando as variáveis adequadamente, visando otimizar o Q-Learning no `FrozenLake-v1`. 

In [ ]:
# Carrega parâmetros ótimos obtidos com o Optuna
study = optuna.load_study(storage="sqlite:///optuna_cap07.db",
                          study_name=f"qlearning_{ENV_NAME_EXPERIMENT}")

print(f"MELHORES PARÂMETROS PARA {ENV_NAME_EXPERIMENT}:")
print(study.best_params)

qlearn_params_f = study.best_params

Ou descomente a linha abaixo, para rodar com bons parâmetros previamente obtidos com o `optuna`.

In [ ]:
#qlearn_params_f = {'lr': 0.22751008394057, 'epsilon': 0.07783149989309379, 'gamma': 0.9763490337420356}

In [ ]:
environment = gym.make("FrozenLake-v1")
EPISODES = 5_000
RUNS = 15
AUTO_LOAD = True

results_f = []

results_f.append( repeated_exec(RUNS, f"Q-Learning ", run_qlearning, environment, EPISODES, **qlearn_params_f, auto_load=AUTO_LOAD) )
clear_output()

#plan_steps = 1
#results_f.append( repeated_exec(RUNS, f"Dyna-Q ({plan_steps} passos)", run_dyna_q, environment, EPISODES, **qlearn_params_f, planning_steps=plan_steps, auto_load=AUTO_LOAD) )
#clear_output()

plan_steps = 5
results_f.append( repeated_exec(RUNS, f"Dyna-Q ({plan_steps} passos)", run_dyna_q, environment, EPISODES, **qlearn_params_f, planning_steps=plan_steps, auto_load=AUTO_LOAD) )
clear_output()

In [ ]:
plot_multiple_results(results_f, cumulative='avg', x_log_scale=False)

In [ ]:
plot_multiple_results(results_f, cumulative='no', x_log_scale=False, window=100)

Analisando os resultados, o Dyna-Q foi bem no ambiente `FrozenLake-v1`?

Apesar de variar entre execuções, ele não costuma ir bem com quantidades altas de "passos de planejamento" neste ambiente.

**Você acha que isso acontece por qual motivo?**

## 5 - Desafio

Proponha uma modificação simples no *modelo* e no *planejamento* do Dyna-Q para melhorar o desempenho dele no `FrozenLake`.

Depois, refaça os experimentos da seção 4.2 para conferir se deu certo.

In [ ]:
def planning_new(model, planning_steps, Q, lr, gamma):
    # altere aqui !!!
    pass

In [ ]:
# Algoritmo Dyna Q
def run_dyna_q_new(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, planning_steps=5):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

   # inicializa a tabela Q
    Q = np.zeros(shape=(env.observation_space.n, num_actions))

    model = dict({})

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state = env.reset()

        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:

            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy_random_tiebreak(Q, state, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)

            if done:
                # para estados terminais
                V_next_state = 0
                next_state = env.reset()
            else:
                # para estados não-terminais -- valor máximo (melhor ação)
                V_next_state = np.max(Q[next_state])

            # atualiza a Q-table / direct RL
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            # atualiza o modelo
            # ALTERE AQUI !!!

            # planejamento / indirect RL
            planning_new(model, planning_steps, Q, lr, gamma)

            sum_rewards += reward
            state = next_state

        sum_rewards_per_ep.append(sum_rewards)

    state = env.reset()
    reward = 0

    return sum_rewards_per_ep, Q